In [133]:
#Import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import timedelta
from sklearn.preprocessing import FunctionTransformer

import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Loading and pre-processing the data**

In [161]:
#Load in the resampled dataset
df = pd.read_csv('data/processed/hourly_pollutionpatterns.csv').set_index('DATETIME')

# Including harmonic signals
def sin_encoder(period):
        return FunctionTransformer(lambda x: np.sin((2 * np.pi* x)/period))

def cos_encoder(period):

    return FunctionTransformer(lambda x: np.cos((2 * np.pi* x)/period))

def adding_remaining_features(df):

    df.index = pd.to_datetime(df.index, utc=True)

    pollutants = ["TOC", "TN", "TP", "SS"]

    df["sine"] = sin_encoder(24).fit_transform(df.index.hour)
    df["cosine"] = cos_encoder(24).fit_transform(df.index.hour)

    # Including day of week
    df["is_weekend"] = df.index.dayofweek.isin([5, 6]).astype(int)

    # Taking the logarithm of Pollutants
    logcn = lambda x: f"log{x}"
    logpollutants = []
    for p in pollutants:
        cn = logcn(p)
        df[cn] = np.log(1 + df[p].divide(df[p].max()))
        logpollutants.append(cn)

    return df

#The function
def dataProcessing_Chunks(inputdf,features,hours_ahead,hours_behind):
    for feature in features:
        for i in range(1,hours_ahead+1,1):
            inputdf[f'{feature}+{i}'] = inputdf[f'{feature}'].shift(-i)
    for feature in features:
        for i in range(1,hours_behind+1,1):
            inputdf[f'{feature}-{i}'] = inputdf[f'{feature}'].shift(i)   
    inputdf.dropna(inplace=True)
    return inputdf
  
#Use the function
fetures_shift = ['TOC','TN','TP','SS','Rain']
df_chunks = dataProcessing_Chunks(df,fetures_shift,6,6)

Naive approach - Prediction is only a monthly average of the value for that day of the week and time of the day

In [158]:
pollutants = ["TOC", "TN", "TP", "SS"]
flows = ["Flow", "Bypass"]
Rain = ["Rain"]
time = ['wd','hr']

In [162]:
#We define the day of the week and hour of the day
df_chunks['DATETIME'] = pd.to_datetime(df_chunks.index)
df_chunks['wd'] = df_chunks['DATETIME'].apply(lambda time: time.dayofweek)
df_chunks['hr'] = df_chunks['DATETIME'].apply(lambda time: time.hour)
df_chunks['m'] = df_chunks['DATETIME'].apply(lambda time: time.month)

In [ ]:
#Rolling window - mean for that time of the day and for that day of the week for the last month
Y = df_chunks[pollutants+time]
for pollutant in pollutants:
    Y[pollutant] = Y.sort_index().groupby([Y['wd'], Y['hr']])[pollutant].transform(lambda x: x.rolling(4, 4).mean())
 
Y.drop(time, axis=1, inplace=True)
Y.fillna(0,inplace=True)

In [178]:
#Plotting the results against the measured values:

pollutant = 'TP'

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_chunks.index, y=df_chunks[pollutant], mode='lines',name='Measured'))
fig.add_trace(go.Scatter(x=Y.index, y=Y[pollutant], mode='lines',name='Forecast'))

fig.show()

In [179]:
#This is not a great match but hey, let's look at some metrics
from sklearn.metrics import mean_squared_error,  r2_score, root_mean_squared_error

for pollutant in pollutants:
    rmse = root_mean_squared_error(df_chunks[pollutant], Y[pollutant])
    mse = mean_squared_error(df_chunks[pollutant], Y[pollutant])
    r2 = r2_score(df_chunks[pollutant], Y[pollutant])
    print(f'{pollutant}: mse={mse}, rmse={rmse}, r2={r2}')


TOC: mse=3027.0577666233403, rmse=55.0187037890147, r2=0.3810577402866696
TN: mse=212.99545084417113, rmse=14.594363666983606, r2=0.37819854071461245
TP: mse=61.29765226032137, rmse=7.8292817205872325, r2=0.6477452037706282
SS: mse=0.49712525527831586, rmse=0.7050710994490668, r2=0.15547475831196078


In [180]:
df_chunks[pollutants].describe()

,TOC,TN,TP,SS
count,26683.000000,26683.000000,26683.000000,26683.000000
mean,116.777905,32.689033,8.221654,0.530561
std,69.934814,18.508338,13.191725,0.767246
min,0.000000,0.000000,0.000000,0.005029
25%,69.383333,20.340286,2.280214,0.221351
50%,111.926042,31.743541,5.170813,0.369937
75%,157.851042,44.092526,8.520823,0.595217
max,999.880208,351.205317,70.076272,10.534854
